MAT555E - Spring 2022 

Cem Sanga - 509201104 -  sanga@itu.edu.tr

Homework 1

In [108]:
import numpy as np
import scipy.integrate as spint

***

# Question 1

Write a python function that takes 4 arguments `a`, `b`, `N` and `fn` where 

* `a` and `b` determine an interval in $\mathbb{R}$ 
* `N` is a positive integer and 
* `fn` is a real valued function

The function you write should calculate the Riemann sum of the function `fn` over the interval [`a`,`b`] with `N` equal subintervals.

# Solution 1

Riemann sums are the discrete forms of the usual integrals. Instead of calculating integrand function along infinitesimaly close x points, we will approximate the intergal with areas of the rectangles. There are types of this implementation which depends on where the argument of the integrand function resides on the respective rectangle. I will use the midpoint convention here. My list of tasks will be;

- Defining a lambda function with variable x for whatever function we want to calculate.
- Using the given endpoints $a$ and $b$, I will get a range for the summation. With this range defined, I will divide it into $N$ sub-intervals to have N rectangles. I will use `np.linspace` for this and I will have $N$ amount of $x_n$ points 
- Area of the rectangles will be calculated. Since we are using midpoint rule, the vertical dimension will be $f(x_{n,mid})$ and the horizontal dimension will be $x_{n+1} - x_{n}$. $x_{n,mid}$ is going to be the average of the two consecutive x values such as $x_{n,mid} = \frac{x_{n+1} + x_{n}}{2}$

In [156]:
a=1
b=100
N=10000
fn = lambda x : 3*(x**2) + 1/x   

In [157]:
def RiemannSum(a,b,N,fn):
    d = (b-a)/N
    x = np.linspace(a,b,N,endpoint=True)
    incr = (x[2]-x[1])/2 # Any two index would do since the intervals are even for any pair.
    xmid = x[:-1] + incr # Excluding last index of x with x[:-1] since there is no point beyond number b.
    rsum=[]
    for n in range(N-1):
        area = fn(xmid[n])*(x[n+1]-x[n])
        rsum.append(area)
    return sum(rsum)

### Reality Check

Now I will use standard scipy library to calculate the same integral and compare the results.

In [162]:
RS = RiemannSum(a,b,N,fn)
integ = spint.quad(fn,a,b)[0]

print("Result with the Riemann sum is   = ", RS)
print("Result with built-in integration = ", integ)

Result with the Riemann sum is   =  1000003.6027398676
Result with built-in integration =  1000003.6051707028


***

# Question 2

We want to sample random samples from the unit circle $x^2+y^2=1$. Write a python function which takes one parameter $N$ the number of sample points and returns a sample $N$ points on the unit circle as a $N\times 2$ numpy matrix. Calculate some examples, and visualize them.

***
# Question 3

Generalize the function above to sampling over the unit sphere $x^2+y^2+z^2=1$. The function should return a $N\times 3$ numpy array.  Calculate some examples, and visualize them. [Hint: Use spherical coordinates.]

***
# Question 4

How can we generalize this to sampling over an $n$-dimensional sphere $x_1^2+\cdots+x_n^2=1$? Write a python function and explain your code.

***
# Question 5

Istanbul Municipality has a [data service](https://data.ibb.gov.tr/) that is accessible via an API. Your task is to visualize the stops. Use the [Basemap](https://matplotlib.org/basemap/) library to draw the map.  Their locations (longitudes and latitudes) are given at this [link](https://data.ibb.gov.tr/dataset/public-transport-gtfs-data/resource/d1f7c258-bbc1-406f-9ab2-7a7c1797c673). Here is a list of things I'd like:

1. You should pull the data via the API. I don't want to see the file locally.
2. In the visualization, the size of the dot representing each stop should depend on the number of people using it.

For the second part, browse through the data server and find the relevant data source.

# Sources

- [Mathematical Python - Riemann Sums](https://personal.math.ubc.ca/~pwalls/math-python/integration/riemann-sums/)
- [N-spheres sampling](http://extremelearning.com.au/how-to-generate-uniformly-random-points-on-n-spheres-and-n-balls/)
-
-